In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes, Hyphae
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips
from realign import transform_skeleton_final


In [2]:
plate = 3
begin = 0
end = 20
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp = pickle.load( open(f'/scratch/shared/mrozemul/Fiji.app/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

In [3]:
def get_hyphae(experiment,exclude_bottom_factor=0.98):
    tips = [node for node in experiment.nodes if node.degree(node.ts()[0])==1 and node.pos(node.ts()[0])[0]<=experiment.boundaries_x[1]*exclude_bottom_factor]
    problems=[]
    small_problems = []
    hyphaes=[]
#     for i in range(20):
    for i,tip in enumerate(tips):
        if i%200==0:
            print(i/len(tips))
#         tip = choice(tips)
        hyphae = Hyphae(tip)
        roots = []
        for t in tip.ts():
#             print(t,tip)
            if tip.degree(t)==1:
                root,edges,nodes = hyphae.get_edges(t,100)
                roots.append(root)
        occurence_count = Counter(roots)
        if len(occurence_count.values())>=2:
            small_problems.append(tip)
        if len(occurence_count.values())>=2 and occurence_count.most_common(2)[0][0]!=roots[0] and occurence_count.most_common(2)[1][1]/occurence_count.most_common(2)[0][1]>=0.75:
            problems.append(tip)
        else:
            hyphae.root = occurence_count.most_common(2)[0][0]
            hyphae.ts = sorted(set(hyphae.ts).intersection(set(hyphae.root.ts())))
            hyphaes.append(hyphae)
    print(f'Detected problems during hyphae detection, {len(small_problems)} hyphaes have inconsistent root over time')
    return(hyphaes,problems)

In [10]:
hyphaes,problems = get_hyphae(exp)

0.0
0.028855864954552012
0.057711729909104025
0.08656759486365603
0.11542345981820805
0.14427932477276006
0.17313518972731207
0.2019910546818641
0.2308469196364161
0.2597027845909681
0.28855864954552013
0.31741451450007213
7 7155 352
7 7155 850
7 7155 989
7 7155 8322
7 7155 9306
7 7155 199
7 7155 8062
7 7155 203
7 7155 352
7 7155 850
7 7155 989
0.34627037945462413
0.3751262444091762
0.4039821093637282
0.4328379743182802
10 10189 11190
10 10189 352
10 10189 850
10 10189 989
10 10189 9390
10 10189 12224
10 10189 10173
10 10189 203
10 10189 11190
10 10189 352
10 10189 850
0.4616938392728322
0.4905497042273842
0.5194055691819363
0.5482614341364882
0.5771172990910403
0.6059731640455923
0.6348290290001443
18 15651 352
18 15651 850
18 15651 989
18 15651 20736
18 15651 9390
18 15651 22251
18 15651 13264
18 15651 203
18 15651 352
18 15651 850
18 15651 989
0.6636848939546963
0.6925407589092483
0.7213966238638003
0.7502524888183524
0.7791083537729043
0.8079642187274564
0.8368200836820083
0.865675

In [11]:
len(hyphaes)

6832

In [42]:
experiment = exp
tips = [node for node in experiment.nodes if node.degree(node.ts()[0])==1 and node.pos(node.ts()[0])[0]<=experiment.boundaries_x[1]*0.98]
problems=[]
small_problems = []
hyphaes=[]
tip =choice(tips)
hyphae = Hyphae(tip)
roots = []
for t in tip.ts():
#             print(t,tip)
    if tip.degree(t)==1:
        root,edges,nodes = hyphae.get_edges(t,100)
        roots.append(root)

In [43]:
roots

[Node(846),
 Node(846),
 Node(846),
 Node(5252),
 Node(5252),
 Node(846),
 Node(8260),
 Node(8260),
 Node(846),
 Node(846),
 Node(846),
 Node(13475),
 Node(846),
 Node(846),
 Node(8260),
 Node(846),
 Node(8260)]

In [44]:
plt.close('all')
begin=4
end=8
exp.plot(tip.ts()[begin:end],[[node.label,tip.label] for node in roots][begin:end])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
tip.show_source_image(6,6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …